## Example showing the features script in the credit_risk_analysis folder

In [1]:
from credit_risk_analysis import features

2025-06-28 22:41:27.367 | INFO     | credit_risk_analysis.config:<module>:11 - PROJ_ROOT path is: C:\Users\andre\iCloudDrive\Documents_Backup\Digital_Bullet_Journal\Education\Data Science\Anyone AI\5. Final_Project\credit_risk_analysis


In [2]:
features.main()

Iniciando procesamiento de características...
Datos cargados desde 'data/interim/X_train.csv', 'X_val.csv', 'y_train.csv' y 'y_val.csv'.
Columnas eliminadas de X_train y X_val.
Datos procesados con el preprocessor (pipeline -> imputer + scaler/encoder).
Columnas finales guardadas en 'data/interim/final_columns.json'.
✅ Preprocessor guardado como 'models/preprocessor.pkl'.
Archivos procesados guardados en 'data/processed/'.
✅ Archivos procesados y guardados correctamente.
Procesamiento completado.


In [3]:
import pandas as pd

In [4]:
X_train = pd.read_csv('data/interim/X_train.csv')
y_train = pd.read_csv('data/interim/y_train.csv')
X_train_processed = pd.read_csv('data/processed/X_train_processed.csv')

In [5]:
X_train.head()

,ID_CLIENT,CLERK_TYPE,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,QUANT_ADDITIONAL_CARDS,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,EDUCATION_LEVEL1,...,EDUCATION_LEVEL2,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3
0,21851,C,10,0,0,1,F,4,0,0,...,0.0,0,0,0,0,2,N,55,171,171
1,29192,C,5,Web,0,1,M,1,0,0,...,NaN,0,0,0,0,1,N,23,473,473
2,45176,C,10,Web,0,1,M,1,0,0,...,NaN,0,0,0,0,1,N,27,726,726
3,17892,C,10,0,0,1,M,2,0,0,...,0.0,0,0,0,0,1,N,57,590,590
4,7513,C,10,Web,0,1,F,1,0,0,...,NaN,0,0,0,0,1,N,50,235,235


In [6]:
X_train_processed.head()

,PERSONAL_MONTHLY_INCOME,OTHER_INCOMES,PERSONAL_ASSETS_VALUE,MARITAL_STATUS,QUANT_DEPENDANTS,MONTHS_IN_RESIDENCE,QUANT_BANKING_ACCOUNTS,QUANT_CARS,AGE,PROFESSION_CODE,...,PRODUCT_1,PRODUCT_2,PRODUCT_7,CITY_OF_BIRTH,RESIDENCIAL_CITY,RESIDENCIAL_BOROUGH,RESIDENCIAL_PHONE_AREA_CODE,PROFESSIONAL_PHONE_AREA_CODE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3
0,-0.024955,-0.036265,-0.064454,1.402019,-0.541334,-0.920372,1.340453,1.408374,0.782999,-2.760757,...,0.0,1.0,0.0,0.000075,0.001425,0.000175,0.163125,0.732050,0.001525,0.001525
1,-0.063755,-0.036265,-0.064454,-0.868794,-0.541334,-0.430826,-0.744233,-0.710039,-1.351917,0.265246,...,1.0,0.0,0.0,0.001200,0.001950,0.000050,0.005000,0.732050,0.002025,0.002025
2,-0.000974,-0.036265,-0.064454,-0.868794,-0.541334,0.058721,-0.744233,-0.710039,-1.085053,0.265246,...,1.0,0.0,0.0,0.000050,0.000525,0.000025,0.020525,0.004900,0.004525,0.004525
3,0.175244,-0.036265,-0.064454,-0.111857,-0.541334,-0.430826,1.340453,1.408374,0.916431,0.265246,...,1.0,0.0,0.0,0.006125,0.007550,0.000125,0.032200,0.008725,0.008150,0.008150
4,-0.064833,-0.036265,-0.064454,-0.868794,-0.541334,-0.332916,-0.744233,-0.710039,0.449418,0.265246,...,1.0,0.0,0.0,0.023400,0.009750,0.001175,0.026325,0.732050,0.002450,0.002450
